In [ ]:
!pip install torch transformers peft bitsandbytes accelerate PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 26.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig

model_name="meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

b_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
import PyPDF2
import re
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    textp = preprocess_text(text)
    return textp

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [ ]:
import torch
tokenizer.pad_token = tokenizer.eos_token

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, text, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.input_ids = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=max_length).input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'labels': self.input_ids[idx]}

file_path = '/content/Machine Learning_ A Visual Starter Course.pdf'
text = extract_text_from_pdf(file_path)
dataset = CustomDataset(text, tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import logging
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
)

# Apply PEFT to the model
peft_model = get_peft_model(b_model, peft_config)

# Training Arguments
logging.basicConfig(level=logging.INFO)

# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=5,
#     per_device_train_batch_size=16,
#     save_steps=500,
#     save_total_limit=2,
#     logging_dir='./logs',
#     logging_steps=100,  # Log every 10 steps
#     report_to="none",
#     learning_rate=5e-5,
#     optim="paged_adamw_8bit",
#     logging_strategy="epoch",  # Ensure logging strategy is set to steps
#     weight_decay=0.01,
# )


#2
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    report_to="none",
    learning_rate=5e-5,
    optim="paged_adamw_8bit",
    logging_strategy="epoch",  # Log at the end of each epoch
    weight_decay=0.01,
)

# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=5,
#     per_device_train_batch_size=4,
#     save_steps=20,
#     save_total_limit=2,
#     logging_dir='./logs',
#     logging_steps=2,
#     learning_rate=5e-5,
#     report_to="none",
#     logging_strategy="epoch"
# )
# Initialize Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()



Step,Training Loss
1,2.907400
2,2.888900
3,2.870600
4,2.858400
5,2.849000


TrainOutput(global_step=5, training_loss=2.874849033355713, metrics={'train_runtime': 20.6799, 'train_samples_per_second': 0.242, 'train_steps_per_second': 0.242, 'total_flos': 115301815418880.0, 'train_loss': 2.874849033355713, 'epoch': 5.0})

In [ ]:
peft_model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.json')